# Imports

In [31]:
import pandas as pd
import glob
import os
from tqdm import tqdm

# Configs

# Import Data

Used Dataset:
- [WESAD: Multimodal Dataset for Wearable Stress and Affect Detection](https://ubicomp.eti.uni-siegen.de/home/datasets/icmi18/): Exclusively importing the available pickle files from each subject (S2.pkl, S2.pkl, ...) and combining them
- Self-Generated Dataset (ECGPrivate):
    - Device: Bioplux Chest (and Finger) Band
    - Dataframes are modeled following the already implemented pydantic models

In [125]:
def dict_to_dataframe_WESAD(dict):
    df = pd.DataFrame()

    df["ECG"] = dict["signal"]["chest"]["ECG"].flatten()
    df["label"] = dict["label"].flatten()
    df["subject"] = dict["subject"]

    return df

In [139]:
from typing import Dict
def combine_pickles_to_df(pattern, wesad_ecg=False):
    """
    Load multiple pickle files that contain similar pandas DataFrames and combine them into a single DataFrame.

    Args:
        pattern (str): A glob pattern that matches the pickle files to load.
        wesad (bool): Whether to convert dictionary objects to DataFrames (used for WESAD dataset).

    Returns:
        A pandas DataFrame that combines the loaded DataFrames.
    """
    dfs = []
    for file_path in tqdm(glob.glob(pattern)):
        with open(file_path, 'rb') as f:
            try:
                df = pd.read_pickle(f)
            except AttributeError:
                raise ValueError(f"Expected a pandas DataFrame, but got {type(df)}")
            if isinstance(df, Dict) and wesad_ecg:
                df = dict_to_dataframe_WESAD(df)
            dfs.append(df)
    combined_df = pd.concat(dfs, ignore_index=True)
    return combined_df

### WESAD Data

In [137]:
%%time
# load WESAD
df = combine_pickles_to_df('../data/S15.pkl', wesad_ecg=True)

100%|██████████| 1/1 [00:09<00:00,  9.46s/it]

CPU times: total: 6.55 s
Wall time: 9.56 s


In [141]:
df.describe()

,ECG,label
count,3.676400e+06,3.676400e+06
mean,1.106015e-03,1.464774e+00
std,2.885650e-01,1.626752e+00
min,-1.257111e+00,0.000000e+00
25%,-1.009369e-01,0.000000e+00
50%,-1.826477e-02,1.000000e+00
75%,2.069092e-02,2.000000e+00
max,1.499496e+00,7.000000e+00


### Self-generated dataset

# Preprocessing

In [ ]:
#ecgs = np.array(ecg_batch.ecgs)

# Detrend signals
#ecgs -= np.mean(ecgs, axis=1, keepdims=True)